# 風險地圖

In [2]:
import os
import pandas as pd
import folium
import json
import geopandas
import folium
from folium.plugins import MarkerCluster, Search
from json2html import *
data = pd.read_csv("106年火災事件資料.csv")
GeoJsonArr = [file for file in os.listdir("./建築執照資料整理") if file.endswith("geo.json")]
len(GeoJsonArr)

21

In [3]:
GeoJsonArr[1][:3]

'臺中市'

## 統計各區域火災數量 (106年)

In [4]:
data_modified = data[(data["縣市"]!="高雄港") & (data["縣市"]!="臺中港")& (data["縣市"]!="基隆港")]
df = pd.DataFrame()
df["區域"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().index.values)
df["數量"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().values)
df

,區域,數量
0,沙鹿區,401
1,南區,393
2,安南區,381
3,彰化市,350
4,中山區,347
...,...,...
350,牡丹鄉,2
351,永福路,2
352,西區,1
353,鹿谷鄉,1


In [17]:
for 縣市 in GeoJsonArr:
  data_modified = data[data["縣市"]==縣市[:3]]
  df = pd.DataFrame()
  df["區域"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().index.values)
  df["數量"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().values)
  data_car_A1 = pd.read_csv(縣市[:3]+"_A1.csv")
  data_car_A2 = pd.read_csv(縣市[:3]+"_A2.csv")
  data_car_pos_A1 = data_car_A1[["經度","緯度","車種","死亡受傷人數"]]
  data_car_pos_A1 = data_car_A1.dropna(axis=0, how='any')
  data_car_pos_A2 = data_car_A2[["經度","緯度","車種","死亡受傷人數"]]
  data_car_pos_A2 = data_car_A2.dropna(axis=0, how='any')
  Total_map = folium.Map(location=[23.58,121], zoom_start=7)
  marker_cluster = MarkerCluster(name="A1車禍統計").add_to(Total_map)

  for item in data_car_pos_A1.values.tolist():
      popuptext = "<h4>車種:</h4> {}<br><h4>死亡人數:</h4>{}<br>地址: {}".format(item[4],item[3],item[2])
      test = folium.Html(popuptext, script=True)
      popup = folium.Popup(test, max_width=100)
      folium.Marker(
            location=[item[6],item[5]],
            popup = popup,
            icon=folium.Icon(color='darkred', icon="car",
            prefix='fa')).add_to(marker_cluster)
      
  marker_cluster2 = MarkerCluster(name="A2車禍統計").add_to(Total_map)

  for item in data_car_pos_A2.values.tolist():
      popuptext = "<h4>車種:</h4> {}<br><h4>死亡人數:</h4>{}<br>地址: {}".format(item[4],item[3],item[2])
      test = folium.Html(popuptext, script=True)
      popup = folium.Popup(test, max_width=100)
      folium.Marker(
            location=[item[6],item[5]],
            popup = popup,
            icon=folium.Icon(color='darkred',icon="car",
            prefix='fa')).add_to(marker_cluster2)

  
  folium.Choropleth(
      geo_data=縣市[:3]+"_區域界線.json",
      name='火災事件',
      data = df,
      fill_color='YlGnBu',
      columns=["區域","數量"],
      key_on='feature.properties.T_Name',
  ).add_to(Total_map)
 
  with open("./建築執照資料整理/"+縣市, "r") as f:
      JSON_file = json.load(f)
  building= geopandas.read_file("./建築執照資料整理/"+縣市)
  addressgeo = folium.GeoJson(
    JSON_file,
    name='建築位置',
    show=False
  ).add_to(Total_map)

  statesearch = Search(
    layer=addressgeo,
    geom_type='Points',
    placeholder='搜尋地址',
    collapsed=False,
    search_label='地址',
    search_zoom=17,
    weight=3,
    #tooltip=folium.GeoJsonTooltip(fields=["地址"],aliases=["地址"])
  ).add_to(Total_map)

  marker_cluster = MarkerCluster(name="建築標記").add_to(Total_map)
  address_list=[]

  for i in range(len(building)):
      try:
          popuptext = "<h4>{}</h4>{}".format(JSON_file["features"][i]["properties"]["地址"]
                                           ,json2html.convert(json = JSON_file["features"][i]["properties"]["樓層概要"]))
      except:
          popuptext = "<h4>{}</h4>".format(JSON_file["features"][i]["properties"]["地址"])
      test = folium.Html(popuptext, script=True)
      popup = folium.Popup(test, max_width=300,min_width=300)
      folium.Marker(
          popup = popup,
          location=[building["經緯"].values[i].split(",")[0],building["經緯"].values[i].split(",")[1]],
          icon=folium.Icon(color='darkgreen', icon='home', prefix='fa')).add_to(marker_cluster)

  folium.LayerControl().add_to(Total_map)


  Total_map.save(縣市[:3]+'.html')

In [18]:
GeoJsonArr

['桃園市107_geo.json',
 '臺中市107_geo.json',
 '宜蘭縣107_geo.json',
 '澎湖縣106_geo.json',
 '雲林縣107_geo.json',
 '連江縣106_geo.json',
 '臺南市107_geo.json',
 '屏東縣107_geo.json',
 '新北市107_geo.json',
 '嘉義市107_geo.json',
 '苗栗縣107_geo.json',
 '嘉義縣107_geo.json',
 '臺東縣106_geo.json',
 '高雄市107_geo.json',
 '金門縣106_geo.json',
 '彰化縣107_geo.json',
 '新竹市107_geo.json',
 '南投縣107_geo.json',
 '基隆市107_geo.json',
 '花蓮縣106_geo.json',
 '新竹縣107_geo.json']